<a href="https://colab.research.google.com/github/manas003884/repo2/blob/master/modify_iwt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import numpy as np
import pywt
from PIL import Image
from skimage.metrics import peak_signal_noise_ratio, mean_squared_error
from skimage.transform import resize

def iwt_steganography(ci, sm, sk):
    # Perform a single-level Haar wavelet decomposition
    coeffs = pywt.dwt2(ci, 'haar')
    ll, (hl, lh, hh) = coeffs

    # Convert the HH, LH, and HL coefficient matrices into vectors
    hh_vec = hh.flatten()
    lh_vec = lh.flatten()
    hl_vec = hl.flatten()

    # Combine sm and sk into a single string
    message_key = sm + sk

    # Initialize the categories as lists of lists
    categories = [[] for _ in range(7)]

    # Assign each coefficient to a category
    for i in range(len(hh_vec)):
        # Cast the result of np.ceil to an integer
        category_index = int(np.ceil(hh_vec[i] / 255.0))
        categories[category_index].append(i)
    for i in range(len(lh_vec)):
        # Cast the result of np.ceil to an integer
        category_index = int(np.ceil(lh_vec[i] / 255.0))
        categories[category_index].append(i)
    for i in range(len(hl_vec)):
        # Cast the result of np.ceil to an integer
        category_index = int(np.ceil(hl_vec[i] / 255.0))
        categories[category_index].append(i)

    # Embed the secret message bits
    for i in range(7):
        for j in categories[i]:
            if message_key:
                # Get the next bit from the combined message and key
                next_bit = int(message_key[0])
                hh_vec[j] = int(hh_vec[j])  # Convert to integer
                hh_vec[j] = min(max(int(hh_vec[j]) & 254, 0), 255) | next_bit  # Embed the bit into the coefficient
                message_key = message_key[1:]

    # Reshape the HH, LH, and HL coefficient vectors back into matrices
    hh = hh_vec.reshape(hh.shape)
    lh = lh_vec.reshape(lh.shape)
    hl = hl_vec.reshape(hl.shape)

    # Perform the inverse Haar wavelet transform
    stego_image = pywt.idwt2((ll, (hl, lh, hh)), 'haar')

    return stego_image



def extract_message_from_stego_image(stego_image, secret_key):
    # Perform a single-level Haar wavelet decomposition on the stego image
    coeffs = pywt.dwt2(stego_image, 'haar')
    ll, (hl, lh, hh) = coeffs

    # Convert the HH, LH, and HL coefficient matrices into vectors
    hh_vec = hh.flatten()
    lh_vec = lh.flatten()
    hl_vec = hl.flatten()

    # Initialize variables to store the extracted message, its binary representation, and its length
    extracted_message = ""
    binary_message = ""
    message_length = 0

    # Extract the message using the provided secret key
    for i in range(len(hh_vec)):
        if message_length < len(secret_key):
            if secret_key[message_length] == '1':
                binary_message += str(int(hh_vec[i]))
            message_length += 1
        else:
            break

    for i in range(len(lh_vec)):
        if message_length < len(secret_key):
            if secret_key[message_length] == '1':
                binary_message += str(int(lh_vec[i]))
            message_length += 1
        else:
            break

    for i in range(len(hl_vec)):
        if message_length < len(secret_key):
            if secret_key[message_length] == '1':
                binary_message += str(int(hl_vec[i]))
            message_length += 1
        else:
            break

    # Ensure the binary message length is a multiple of 8 (byte alignment)
    padding = 8 - (len(binary_message) % 8)
    if padding < 8:
        binary_message = '0' * padding + binary_message

    # Convert the binary message to a string
    extracted_message = ''.join([chr(int(binary_message[i:i + 8], 2)) for i in range(0, len(binary_message), 8)])

    return extracted_message

def main():
    # Input file paths
    cover_image_path = "/content/earth.png"
    secret_message_txt = "/content/secret_message.txt"
    secret_key = "rand45"

    # Load the cover image from file
    cover_image = np.array(Image.open(cover_image_path))

    # Load the secret message from a .txt file
    with open(secret_message_txt, 'r') as file:
        secret_message = file.read()

    # Convert the secret message and secret key to binary format
    secret_message_binary = ''.join(format(ord(char), '08b') for char in secret_message)
    secret_key_binary = ''.join(format(ord(char), '08b') for char in secret_key)

    # Perform image steganography
    stego_image = iwt_steganography(cover_image, secret_message_binary, secret_key_binary)

    # Ensure stego_image and cover_image have the same dimensions
    if stego_image.shape != cover_image.shape:
        stego_image = resize(stego_image, cover_image.shape, mode='reflect', anti_aliasing=True)

    # Convert the stego image to uint8 format
    stego_image_uint8 = np.uint8(stego_image)

    # Save the stego image to a file
    stego_image_path = "stego_image.png"
    stego_image_pillow = Image.fromarray(stego_image_uint8)
    stego_image_pillow.save(stego_image_path)

    # Calculate PSNR and MSE
    psnr_value = peak_signal_noise_ratio(cover_image, stego_image_uint8)
    mse_value = mean_squared_error(cover_image, stego_image_uint8)

    print(f"PSNR: {psnr_value:.2f} dB")
    print(f"MSE: {mse_value:.2f}")

    # Extract the hidden message from the stego image
    extracted_message = extract_message_from_stego_image(stego_image_uint8, secret_key)

    print(f"Extracted Message: {extracted_message}")

if __name__ == "__main__":
    main()



PSNR: 75.19 dB
MSE: 0.00
Extracted Message: 


In [12]:
import cv2
import numpy as np
def haar_wavelet_transform(gray_image):
    """
    Applies the Haar wavelet transform to a grayscale image.
    Args:
        gray_image (numpy.ndarray): The grayscale image to transform.
    Returns:
        numpy.ndarray: The Haar wavelet coefficients of the image.
    """
    # Initialize the Haar wavelet coefficients image
    haar_image = np.zeros_like(gray_image)
    # Iterate over the image in 2x2 blocks
    for i in range(0, gray_image.shape[0], 2):
        for j in range(0, gray_image.shape[1], 2):
            # Calculate the average and difference values
            avg = (gray_image[i, j] + gray_image[i+1, j] + gray_image[i, j+1] + gray_image[i+1, j+1]) / 4
            diff = (gray_image[i, j] - gray_image[i+1, j] - gray_image[i, j+1] + gray_image[i+1, j+1]) / 4
            # The average value is the low-pass filtered value
            haar_image[i, j] = avg
            # The difference value is the high-pass filtered value
            haar_image[i+1, j] = diff
            haar_image[i, j+1] = diff
            haar_image[i+1, j+1] = -diff
    # Return the Haar wavelet coefficients
    return haar_image
def inverse_haar_wavelet_transform(haar_image):
    """
    Applies the inverse Haar wavelet transform to a grayscale image.
    Args:
        haar_image (numpy.ndarray): The Haar wavelet coefficients of the image.
    Returns:
        numpy.ndarray: The grayscale image.
    """
    # Initialize the grayscale image
    gray_image = np.zeros_like(haar_image)
    # Iterate over the image in 2x2 blocks
    for i in range(0, gray_image.shape[0], 2):
        for j in range(0, gray_image.shape[1], 2):
            # Calculate the average and difference values
            avg = haar_image[i, j]
            diff = haar_image[i+1, j]
            # The average value is the low-pass filtered value
            gray_image[i, j] = avg
            # The difference value is the high-pass filtered value
            gray_image[i+1, j] = avg - diff
            gray_image[i, j+1] = avg - diff
            gray_image[i+1, j+1] = avg + diff
    # Return the grayscale image
    return gray_image
# Read the image
image = cv2.imread('/content/earth.jpg', cv2.IMREAD_GRAYSCALE)
# Apply the Haar wavelet transform
haar_image = haar_wavelet_transform(image)
# Apply the inverse Haar wavelet transform
gray_image = inverse_haar_wavelet_transform(haar_image)
# Convert the image back to color
color_image = cv2.cvtColor(gray_image, cv2.COLOR_GRAY2BGR)
# Display the original and transformed images
cv2.imshow('Original Image', image)
cv2.imshow('Haar Wavelet Transform', haar_image)
cv2.imshow('Inverse Haar Wavelet Transform', gray_image)
cv2.imshow('Color Image', color_image)
# Wait for a key press
cv2.waitKey(0)
# Close the windows
cv2.destroyAllWindows()

<ipython-input-12-683900349973>:18: RuntimeWarning: overflow encountered in ubyte_scalars
  diff = (gray_image[i, j] - gray_image[i+1, j] - gray_image[i, j+1] + gray_image[i+1, j+1]) / 4


IndexError: ignored